In [1]:
%cd ..
%reload_ext autoreload
%autoreload 2

D:\ownCloud\Uni\Semester Ma 5\3D Scanning & Motion Capture (IN2354)\Project\3D-FaceReconstruction


In [2]:
import eos
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import trimesh
import pickle

from xml.dom import minidom
from collections import defaultdict

from face_reconstruction.model import BaselFaceModel
from face_reconstruction.landmarks import save_landmarks

from env import BASEL_FACE_MODEL_PATH

# 1. Load model

In [3]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")

In [4]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [5]:
face_mesh = bfm.draw_sample([0 for _ in range(n_shape_coefficients)], [0 for _ in range(n_expression_coefficients)], [0 for _ in range(n_color_coefficients)])

In [6]:
face_trimesh = bfm.convert_to_trimesh(face_mesh)
#face_trimesh.export(f"{BASEL_FACE_MODEL_PATH}/model2019_face12.ply")

# 2. Load manually picked landmark coordinates from MeshLab file

In [7]:
def parse_xml_point(xml_point):
    point_id = int(xml_point.attributes['name'].value)
    coordinates = np.array([float(xml_point.attributes['x'].value), float(xml_point.attributes['y'].value), float(xml_point.attributes['z'].value)])
    return point_id, coordinates

In [8]:
xml_doc = minidom.parse(f"{BASEL_FACE_MODEL_PATH}/model2019_face12_landmarks.pp")

In [9]:
points = xml_doc.getElementsByTagName("point")
multi_pie_landmark_coordinates = [parse_xml_point(point) for point in points]

In [10]:
multi_pie_landmark_coordinates

[(1, array([-69.931007,  28.560308,  48.61932 ])),
 (2, array([-69.627991 ,   9.4500484,  48.533798 ])),
 (3, array([-65.37912 , -14.477787,  51.418316])),
 (4, array([-61.998131, -33.647957,  52.082237])),
 (5, array([-54.344574, -52.718498,  51.751541])),
 (6, array([-41.773003, -66.850563,  62.176857])),
 (7, array([-27.938793, -78.346947,  64.261902])),
 (8, array([-13.789294, -84.575066,  71.738678])),
 (9, array([  0.19254546, -86.417137  ,  73.450485  ])),
 (10, array([ 18.038544, -83.46743 ,  69.182701])),
 (11, array([ 33.134388, -75.307884,  60.965725])),
 (12, array([ 43.878792, -66.84166 ,  54.778755])),
 (13, array([ 57.802597, -47.262321,  50.653191])),
 (14, array([ 63.404911, -29.730335,  51.239803])),
 (15, array([ 65.886734, -11.012751,  51.095459])),
 (16, array([69.782455 ,  9.7624693, 48.27858  ])),
 (17, array([68.999817, 29.877851, 49.063202])),
 (18, array([-57.599289,  45.260448,  80.311104])),
 (19, array([-51.050247,  50.861427,  90.475594])),
 (20, array([-4

# 3. Find corresponding vertex IDs in mesh

In [11]:
closest_vertices = {}
closest_distances = {}
for vertex_id, vertex in enumerate(face_trimesh.vertices):
    for landmark_idx, landmark in multi_pie_landmark_coordinates:
        distance = np.linalg.norm(landmark - vertex)
        if landmark_idx not in closest_vertices or distance < closest_distances[landmark_idx]:
            closest_vertices[landmark_idx] = vertex_id
            closest_distances[landmark_idx] = distance

In [12]:
closest_vertices

{1: 12550,
 2: 25563,
 3: 11735,
 4: 107,
 5: 2926,
 6: 585,
 7: 26043,
 8: 23314,
 9: 429,
 10: 5494,
 11: 4710,
 12: 361,
 13: 11847,
 14: 18345,
 15: 20967,
 16: 14965,
 17: 26053,
 18: 22083,
 19: 10487,
 20: 27365,
 21: 5909,
 22: 24076,
 23: 23596,
 24: 22244,
 25: 24263,
 26: 24301,
 27: 17422,
 28: 26082,
 29: 24204,
 30: 4580,
 31: 45,
 32: 27273,
 33: 1081,
 34: 24839,
 35: 21702,
 36: 12561,
 37: 6898,
 38: 6740,
 39: 14119,
 40: 16732,
 41: 23309,
 42: 24290,
 43: 20843,
 44: 26598,
 45: 14406,
 46: 5534,
 47: 11246,
 48: 19783,
 49: 11660,
 50: 12568,
 51: 25963,
 52: 1104,
 53: 7951,
 54: 12703,
 55: 4945,
 56: 24720,
 57: 7451,
 58: 22167,
 59: 6942,
 60: 25919,
 61: 15130,
 62: 14765,
 63: 19732,
 64: 2225,
 65: 9718,
 66: 17791,
 67: 13689,
 68: 4140}

In [13]:
save_landmarks(closest_vertices, "model2019_face12_landmarks")

# 4. Visualize outputs

In [14]:
for vertex_id in closest_vertices.values():
    vertex = face_trimesh.visual.vertex_colors[vertex_id]
    vertex[0] = 255
    vertex[1] = 0
    vertex[2] = 0

In [15]:
perspective_camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.414)
directional_light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)

In [16]:
scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(face_trimesh))
scene.add(perspective_camera, pose=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 300], [0, 0, 0, 1]])) # Position camera just in front of face
scene.add(directional_light)

In [17]:
pyrender.Viewer(scene, use_raymond_lighting=True)

Viewer(width=640, height=480)